In [183]:
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import requests
import pandas as pd
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from datetime import datetime
import urllib.request
import dash_table
import dash_table.FormatTemplate as FormatTemplate
from dash_table.Format import Sign
import numpy as np
import ssl
import json
import time


In [184]:
def get_n_top_coins(n=10):
    url = 'https://api.lunarcrush.com/v2?data=market&key=39o5gbbtx7n1gxkvnhf6na&limit={}&sort=acr&type=fast'.format(n)
    response = requests.get(url).json()
    df = pd.json_normalize(response['data'])
    df.rename(columns={
        's': 'Symbol',
        'n': 'Name',
        'p': 'Price',
        'v': 'Volume',
        'pc': '24 Hours',
        'pch': '1 Hour',
        'mc': 'Market Cap',
        'gs': 'Galaxy Score',
        'ss': 'Social Score',
        'acr': 'ALTRank',
        't':'Number of tweets',
        'sd':'Social Dominance',
        'd' : 'Market Dominance'
        }, inplace=True)
    df['24 Hours'] = df['24 Hours'].div(100).round(4)
    df['1 Hour'] = df['1 Hour'].div(100).round(4)

    return df[['ALTRank','Galaxy Score','Social Score','Symbol','Name','Price','Volume','24 Hours','1 Hour','Market Cap','Number of tweets','Social Dominance','Market Dominance']]


In [185]:
def get_market_table(n=50):
    market_data = get_n_top_coins(n)
    market_table = dash_table.DataTable(
        id='market_table',
        data=market_data.to_dict('records'),
        columns=[
            {'id': 'ALTRank', 'name': 'ALTRank', 'type': 'numeric'},
            {'id': 'Galaxy Score', 'name': 'Galaxy Score (PT)',
             'type': 'numeric'},
            {'id': 'Social Score', 'name': 'Social Score (PT)',
             'type': 'numeric'},
            {'id': 'Symbol', 'name': 'Symbol', 'type': 'text'},
            {'id': 'Name', 'name': 'Name', 'type': 'text'},
            {
                'id': 'Price',
                'name': 'Price',
                'type': 'numeric',
                'format': FormatTemplate.money(6),
                },
            {
                'id': 'Volume',
                'name': 'Volume',
                'type': 'numeric',
                'format': FormatTemplate.money(2),
                },
            {
                'id': '24 Hours',
                'name': '24 Hours',
                'type': 'numeric',
                'format': FormatTemplate.percentage(2).sign(Sign.positive),
                },
            {
                'id': '1 Hour',
                'name': '1 Hour',
                'type': 'numeric',
                'format': FormatTemplate.percentage(2).sign(Sign.positive),
                },
            {
                'id': 'Market Cap',
                'name': 'Market Cap',
                'type': 'numeric',
                'format': FormatTemplate.money(0),
                },
            ],
        style_data_conditional=[
    #         data_bars(market_data, 'Volume')[0],
            {'if': {'filter_query': '{24 Hours} > 0',
             'column_id': '24 Hours'}, 'color': 'green',
             'fontWeight': 'bold'},
            {'if': {'filter_query': '{24 Hours} < 0',
             'column_id': '24 Hours'}, 'color': 'red', 'fontWeight': 'bold'
             },
            {'if': {'filter_query': '{24 Hours} = 0',
             'column_id': '24 Hours'}, 'color': 'blue', 'fontWeight': 'bold'
             },
            {'if': {'filter_query': '{1 Hour} > 0', 'column_id': '1 Hour'},
             'color': 'green', 'fontWeight': 'bold'},
            {'if': {'filter_query': '{1 Hour} < 0', 'column_id': '1 Hour'},
             'color': 'red', 'fontWeight': 'bold'},
            {'if': {'filter_query': '{1 Hour} = 0', 'column_id': '1 Hour'},
             'color': 'blue', 'fontWeight': 'bold'},
            {'if': {'row_index': 'odd'},
             'backgroundColor': '#061e44'},
            ],
        style_cell_conditional=[{'if': {'column_id': 'Name'},
                                'textAlign': 'left'}, 
    #                             {
    #         'if': {'column_id': 'Volume'},
    #         'width': '200px',
    #         'color': 'blue',
    #         'minWidth': '200px',
    #         'maxWidth': '200px',
    #         'overflow': 'hidden',
    #         'textOverflow': 'ellipsis',}, 
                                {'if': {'column_id': ('Symbol','ALTRank','Galaxy Score')}, 
                                 'fontWeight': 'bold',
                                 'textAlign': 'center'}],
        style_header={'backgroundColor': '#13326c',
                      'fontWeight': 'bold',
                      'fontSize':17,
                     },
        style_cell = {'fontSize':15,
                      'font-family':'Lucida Console',
                      'backgroundColor':"#082255",
                      'color':'#e5e9f0',
                      'border':'1px solid #082255'},
        # jakiś error podczas sortowania np 24 hours lub 1houur
        sort_action='native',
        editable=False,
        style_as_list_view=True,
        page_size=5,
    #     fill_width=False
        )
    return market_table

In [186]:
def get_coin_data(symbol,data_points=365,interval='day'):
    url = "https://api.lunarcrush.com/v2?data=assets&key=39o5gbbtx7n1gxkvnhf6na&symbol={}&data_points={}&interval={}".format(symbol,data_points,interval)
    response = requests.get(url).json()
    data = response['data'][0]
    df = pd.json_normalize(data['timeSeries'])
    df['time'] = pd.to_datetime(df['time'],unit='s')
    data = pd.json_normalize(data)
    details = data[['name','symbol','price_btc','market_cap','percent_change_24h','percent_change_7d','percent_change_30d','volume_24h','max_supply']]

    return (df,details.to_dict(orient='records')[0])

In [187]:
x = get_coin_data('BTC',data_points=365,interval='day')
x = pd.DataFrame.from_dict(x[0])
x

,asset_id,time,open,close,high,low,volume,market_cap,url_shares,unique_url_shares,...,medium,youtube,social_contributors,social_volume,price_btc,social_volume_global,social_dominance,market_cap_global,market_dominance,percent_change_24h
0,1,2020-05-07,9339.670316,9972.584023,9972.584023,9339.670316,66200111108,1.720972e+11,39758,23668,...,30.0,56.0,28758,193886,1,349379,55.494463,272568627918,63.139021,9.183341
1,1,2020-05-08,9942.673964,9844.867939,9942.673964,9833.070782,57650692921,1.810168e+11,37721,22187,...,25.0,48.0,36141,160423,1,333372,48.121318,270067625347,67.026478,-1.280672
2,1,2020-05-09,9829.918495,9574.098730,9829.918495,9574.098730,48237707683,1.779054e+11,37514,19693,...,24.0,63.0,28251,162925,1,290032,56.174836,265101113579,67.108500,-2.750359
3,1,2020-05-10,9260.257076,8750.250756,9260.257076,8355.023315,69141896093,1.604725e+11,37403,20559,...,26.0,67.0,34522,159492,1,270693,58.919883,244002332325,65.766781,-8.604966
4,1,2020-05-11,8708.713290,8610.172121,9010.995404,8395.370859,63796473623,1.637024e+11,54206,29210,...,39.0,77.0,31069,281147,1,406627,69.141252,240427603037,68.088031,-1.600853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,1,2021-05-02,57910.855965,56701.923053,57910.855965,56701.923053,45106194800,1.076592e+12,45871,19224,...,1.0,5.0,161842,222439,1,995999,22.333255,2297252739235,46.864309,-1.945530
361,1,2021-05-03,59821.271447,57333.986544,59920.058393,57333.986544,61352700656,1.075031e+12,53420,22946,...,6.0,7.0,155417,251654,1,1204465,20.893426,2388467421328,45.009229,1.114713
362,1,2021-05-04,56420.250356,53422.379991,56420.250356,53422.379991,80573889830,1.041267e+12,45176,23663,...,4.0,12.0,143668,259449,1,1453028,17.855747,2267388525679,45.923609,-6.822492
363,1,2021-05-05,54594.895265,57524.231735,57524.231735,54594.895265,87532745708,1.020221e+12,44439,23320,...,4.0,5.0,173059,291791,1,1596123,18.281235,2471362008024,41.281740,7.678152


In [188]:
fig = go.Figure(go.Indicator(
    mode = "gauge+number",
    value = 450,
    title = {'text': "Price"},
    domain = {'x': [0, 1], 'y': [0, 1]}
))

fig.show()

In [189]:
def build_header():
    return html.Div([
        html.Div([ 
            html.H4("Our Title", className = 'app-header-title'),
            html.P("Some descriptons, maybe our names or somethings like this", 'app-header-subtitle'),
        ], className = 'app-header-main'),
        html.Img(
            src = 'assets/crunchips.png',
            className = 'logo'
        ),
        html.Audio(id = 'easter-egg', controls=True, autoPlay = True, hidden = True)
    ], className = 'app-header')

In [198]:
def build_tabs():
    return html.Div(
        id = 'tabs',
        className="tabs",
        children = [
            dcc.Tabs(
                id = 'control-tabs',
                value = 'tab1',
                className="custom-tabs",
                children=[
                    dcc.Tab(
                        id = 'Main-tab',
                        label = 'Main Tab',
                        value = "tab1",
                        className="custom-tabs",
                        selected_className="custom-tab--selected",
                    ),
                    dcc.Tab(
                        id = 'Sub-tab',
                        label = 'Sub Tab',
                        value = "tab2",
                        className="custom-tabs",
                        selected_className="custom-tab--selected",
                    ),
                    dcc.Tab(
                        id = 'Rest-tab',
                        label = 'Rest Tab',
                        value = "tab3",
                        className="custom-tabs",
                        selected_className="custom-tab--selected",
                    ),
                ],
            )
        ],
    )

In [191]:
def get_social_coin_data(symbol):
    url = 'https://api.lunarcrush.com/v2?data=assets&key=39o5gbbtx7n1gxkvnhf6na&symbol={}'.format(symbol)
    response = requests.get(url).json()['data']

    df = pd.json_normalize(
                    response, 
                    record_path =['timeSeries'], 
                    meta=['symbol']
                            )
    df['time'] = pd.to_datetime(df['time'],unit='s')
    df = df[['unique_url_shares','url_shares','reddit_posts','tweets','tweet_spam','time','symbol']]
    return df

In [192]:
z  = get_n_top_coins(n=50)
coins = z['Symbol'].to_list()


In [193]:
options = []
for coin in coins:
    options.append({'label': coin, 'value': coin})


In [194]:
def build_tab_1():
    return [
        html.Div([
            html.Div([get_market_table(),
                     ],
                     className = 'tabble-wrapper',
                    ),
            
            html.Div([html.Div([html.P("Type:"),
                                dcc.Dropdown(id='types',
                                             className = 'dropdown-type',
                                             value='Market Dominance',
                                             options=[{'label': x,
                                                       'value': x,
                                                      } for x in ['Market Dominance',
                                                                  'Social Dominance',
                                                                  'Number of tweets']],
                                             clearable=False,
                                             # trzeba pozmieniac jakos na naturalne kolory te dropdowny 
                                             style={ 
                                                    'font-family':'Lucida Console',
                                                    'color':'#e5e9f0',
                                                    'border':'0px',
                                                    
                                                   }),
                                html.P("Enter a stock symbol:"),
                                dcc.Dropdown(id='selected_coins',
                                             className = 'dropdown-top',
                                             value=coins[:3],
                                             options=options,
                                             multi = True, 
                                             style={'color': '#e5e9f0',
                                                    }),
                               ], 
                               className = 'dropdowns-wrapper',
                               # trzeba pozmieniac jakos na naturalne kolory te dropdowny 
                              style={
                                  'color': '#e5e9f0',
                                  'font-family':'Lucida Console',
                                  
                              }),
                      html.Div([dcc.Graph(id="pie-chart")],
                               className = 'gauge-wrapper'),

                html.Div([
                    dcc.Graph(id="scatter-chart")
                ], className = 'scatter-wrapper')
            ], className = 'bottom-wrapper')
            
        ], className = 'first-div')
    ]

In [195]:
def build_tab_2():
    return [
        html.Div([
            html.Div([
                html.Div([
                    html.Label("Coin"),
                    dcc.Dropdown(
                          id='coin-dropdown',
                          clearable=False,
                          value='BTC',
                          options=[{'label': c, 'value': c} for c in coins],
                            style={'color': '#212121'})
                ],className = 'dropdowns-wrapper'),
                html.Div([
                    dcc.Graph(id='graph')
                ], className = 'plotline-wrapper'),
                html.Div([
                    dcc.Graph(id='graph-social')
                ], className = 'plotline-wrapper')
                
            ], className = 'top-wrapper')
        ], className = 'second-div')
    ]

In [196]:
def build_tab_3():
    return []

In [197]:
app = JupyterDash(__name__)
app.layout = html.Div([
    html.Div([
        build_header(),
        build_tabs(),
        html.Div(id = 'app-content')
    ], style = {'padding': 20})
    
])
@app.callback(
    Output('app-content', 'children'),
    Input('control-tabs', 'value')
)
def prepare_top(tab_switch):
    if tab_switch == 'tab1':
        return build_tab_1()
    elif tab_switch == 'tab2':
        return build_tab_2()
    else:
        return build_tab_3()

@app.callback(
    [Output("pie-chart", "figure"),
     Output("scatter-chart","figure")], 
    [Input("types", "value"),
    Input('selected_coins','value')])
def generate_chart(types,selected_coins):
    df = get_n_top_coins(50)
    df = df[df['Symbol'].isin(selected_coins)]
    fig_pie = px.pie(df,
                     values=df[types],
                     names=df['Symbol'],
                     hole=+.5,
                     title=types,
                    color_discrete_sequence= px.colors.sequential.Plasma_r)
    
    fig_pie.update_layout(plot_bgcolor = '#082255', paper_bgcolor = '#082255', font_color = '#FFFFFF', width = 400)
    
    fig_scatter = px.scatter(df,
                             x=df['Price'],
                             y=df['Volume'],
                             size=df[types],
                             color=df['Symbol'],
                             hover_name=df['Symbol'],
                             log_y=True,
                             log_x=True,
                             size_max=100,
                             color_discrete_sequence= px.colors.sequential.Plasma_r)
    fig_scatter.update_layout(title=types,
                             xaxis_title="Price ($)",
                             yaxis_title="Volume($)",
                             plot_bgcolor='#082255',
                             )
    
    fig_scatter.update_layout( paper_bgcolor = '#082255', 
                              font_color = '#FFFFFF',
                              xaxis=dict(showgrid=False),
                             yaxis=dict( gridcolor="#0f41a3"),
                             )
    
    return fig_pie,fig_scatter
# @app.callback(
#     Output('easter-egg', 'src'),
#     Input('coin-dropdown', 'value')
# )
# def easteregg(coin):
#     if coin =='DOGE':
#         return 'assets/dog.wav'
#     else:
#         return ''
@app.callback(
    [Output('graph', 'figure'),
     Output('graph-social', 'figure')] ,
    [Input('coin-dropdown', 'value')]
)
def update_figure(coin):
    x, details = get_coin_data(coin)
    x['MA-Low'] = x.low.rolling(window=20).mean()
    x['MA-High'] = x.high.rolling(window=20).mean()
    fig = go.Figure(data=[go.Candlestick(x=x['time'],
                                         open=x['open'],
                                         high=x['high'],
                                         low=x['low'],
                                         close=x['close'],
                                         name='{} price'.format(details['symbol']))])
    
    fig.update_layout(title=details['name'],
                      yaxis_tickformat='$',
                      xaxis_title='Date',
                      yaxis_title='Price',
                     )
    
    fig.update_xaxes(title_text='Date',
                     rangeslider_visible=True,
                     rangeselector=dict(buttons=list([dict(count=1,
                                                           label='1M',
                                                           step='month',
                                                           stepmode='backward'),
                                                      dict(count=6,
                                                           label='6M',
                                                           step='month',
                                                           stepmode='backward'),
                                                      dict(count=1,
                                                           label='YTD',
                                                           step='year', 
                                                           stepmode='todate'),
                                                      dict(count=1,
                                                           label='1Y', 
                                                           step='year', 
                                                           stepmode='backward'),
                                                      dict(step='all')]), bgcolor='black'))
    
    fig.add_trace(go.Scatter(x=x['time'],
                             y=x['MA-Low'],
                             mode='lines',
                             name='Low-Moving Average',
                             line={'color':'#ff9100',
                                  'width':1}))
    fig.add_trace(go.Scatter(x=x['time'],
                             y=x['MA-High'],
                             mode='lines',
                             name='High-Moving Average',
                             line={'color':'#006eff',
                                  'width':1}))
    fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='#e5e9f0',
        ),
        ),
        yaxis=dict(
            showgrid=True,
            zeroline=False,
            showline=True,
            showticklabels=True,
            gridcolor="#0f41a3",
            color='#e5e9f0'
    ),
        showlegend=True,
        plot_bgcolor='#082255',
        xaxis_rangeslider_visible=False
    )
    fig.update_layout( paper_bgcolor = '#082255', font_color = '#FFFFFF', width = 1100)
    
    df = get_social_coin_data(coin)
    mode_size = [8, 8, 8, 8]
    colors = px.colors.sequential.Plasma_r
    fig1 = go.Figure()
    labels = ['unique_url_shares','url_shares','tweets','tweet_spam']
    for j,i in enumerate(labels):
        fig1.add_trace(go.Scatter(x=df['time'], y=df[i], mode='lines',
            name=labels[j],
            line=dict(color=colors[j], width=2),
            connectgaps=True,
        ))
    
    fig1.update_layout(
        xaxis=dict(
            showline=True,
            showgrid=False,
            showticklabels=True,
            linecolor='rgb(204, 204, 204)',
            linewidth=2,
            ticks='outside',
            tickfont=dict(
                family='Lucida Console',
                size=12,
                color='#e5e9f0',
            ),
        ),
        yaxis=dict(
            showgrid=False,
            zeroline=False,
            showline=False,
            showticklabels=True,
        ),
        autosize=False,
        title="24 hours {} social".format(coin),
        xaxis_title="Date",
        yaxis_title="Number",
        showlegend=True,
        plot_bgcolor='#082255',
        paper_bgcolor = '#082255',
        font_color = '#e5e9f0'
    )

    
    
    
    
    
    
    return fig,fig1

app.run_server(mode='external', port = 8054)

Dash app running on http://127.0.0.1:8054/
